#Language Modelling : N-Gram
by : Muhammad Faiz Abdurrahman Djauhar

In [ ]:
!pip install -Uqq fastbook fastai

In [ ]:
from fastbook import *
from fastai.text.all import *
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://raw.githubusercontent.com/Wikidepia/indonesian_datasets/master/crawl/twitter-puisi/data/pelangipuisi.jsonl

--2022-05-31 13:07:22--  https://raw.githubusercontent.com/Wikidepia/indonesian_datasets/master/crawl/twitter-puisi/data/pelangipuisi.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11389849 (11M) [text/plain]
Saving to: ‘pelangipuisi.jsonl.7’

pelangipuisi.jsonl. 100%[===================>]  10.86M  --.-KB/s    in 0.1s    

2022-05-31 13:07:23 (82.7 MB/s) - ‘pelangipuisi.jsonl.7’ saved [11389849/11389849]



In [ ]:
!gdown 1ARRoV6vu0Q6V9wJRHtVcpHXw9A__2FWa

Downloading...
From: https://drive.google.com/uc?id=1ARRoV6vu0Q6V9wJRHtVcpHXw9A__2FWa
To: /content/vocab.json
100% 205k/205k [00:00<00:00, 79.4MB/s]


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

## Data Preparation

In [ ]:
df = pd.read_json(path_or_buf="pelangipuisi.jsonl", lines=True) # read data, convert it to dataframe
df = df.replace(r'\n',' ', regex=True) # replace new line into single space
df = df.applymap(lambda s:s.lower() if type(s) == str else s) # convert all character into lower case
df.head()

,text
0,"hanya karena sapa itu. kau tikam rasamu. sisakan, bulir-bulir sedu."
1,sedang di antrian panjang pada sebuah penantian entah kapan rindu memanggil sebab ada temu yang masih tertinggal
2,"jika kau bukan tempat awal untuk berlabuh, maka kau yang terakhir di mana aku terjatuh kemudian karam"
3,"setiap waktu, aku masih mendengar getar dawai aksaramu, hingga aku terlelap dibuai bayangmu, dan setiap waktuku masih berdo'a yang sama, membelai gelombang tak lupa pula"
4,sebait rindu yang kau bacakan masih terdengar jelas dalam keresahan selembar rindu yang kau tulis di catatan masih terbaca nyata dalam kegilaan sekumpulan rindu yang kau jadikan buku bacaan masih kucetak menjadi kenangan paling edan


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85794 entries, 0 to 85793
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    85794 non-null  object
dtypes: object(1)
memory usage: 670.4+ KB


In [ ]:
df_test = df[85000:]

In [ ]:
batch_size = 64
max_seq_len = 72
seed = 42
val_sz = 0.1

In [ ]:
set_seed(42)
dls = TextDataLoaders.from_df(df[:85000], valid_pct=val_sz, seed=seed, is_lm=True, seq_len=max_seq_len, backwards=False, bs=batch_size)

In [ ]:
dls_test = TextDataLoaders.from_df(
    df_test, valid_pct=0, seed=seed,
    is_lm=True, seq_len=max_seq_len,
    backwards=False, bs=batch_size, text_vocab=dls.vocab
)

In [ ]:
dls.show_batch(max_n=3)

,text,text_
0,"xxbos akan ada kabar dijatuhkan burung , bukan air hujan , tapi mataku basah karenanya . xxbos xxunk , awalnya aku ragu tentang rasa yang tak pernah hilang , ternyata itu benar adanya . kamu pernah pergi , sekarang datang lagi . xxbos ketika kau tanya mengapa aku diam mengapa aku ingin sendiri mengapa aku akan menjawabnya mengapa kau selalu ada ; dalam setiap yang aku lihat … xxbos saat purnama begitu","akan ada kabar dijatuhkan burung , bukan air hujan , tapi mataku basah karenanya . xxbos xxunk , awalnya aku ragu tentang rasa yang tak pernah hilang , ternyata itu benar adanya . kamu pernah pergi , sekarang datang lagi . xxbos ketika kau tanya mengapa aku diam mengapa aku ingin sendiri mengapa aku akan menjawabnya mengapa kau selalu ada ; dalam setiap yang aku lihat … xxbos saat purnama begitu pijar"
1,"lembut nian mengusap nurani menyeka jiwa setenang maut xxbos aku bisa menukar sepi dengan secangkir kopi … tapi bagaimana dengan rindu ini ? xxbos hei xxunk kau ? aku di sini menyanyikan lagu sedu sedan untukmu , ungkapan betapa gersang dunia yang tak kau xxunk cinta … xxbos detik - detik waktu berdenting . xxunk bunga melayang jatuh membumi . pada tanah , semua makhluk pulang , berserah . kembali ; dipeluk","nian mengusap nurani menyeka jiwa setenang maut xxbos aku bisa menukar sepi dengan secangkir kopi … tapi bagaimana dengan rindu ini ? xxbos hei xxunk kau ? aku di sini menyanyikan lagu sedu sedan untukmu , ungkapan betapa gersang dunia yang tak kau xxunk cinta … xxbos detik - detik waktu berdenting . xxunk bunga melayang jatuh membumi . pada tanah , semua makhluk pulang , berserah . kembali ; dipeluk erat"
2,"yang selalu aku perjuangkan sendirian . xxbos aku akan diam , tetapi jemariku akan terus bicara , berteriak tentang keadilan yang kau bungkam . xxbos pada senyum yang sore itu merekah ; sungguh aku tak sengaja jatuh cinta . xxbos kisah lama kembali terkenang lagu lama kembali terngiang nyala lilin mulai surut padam xxbos semesta memberi isyarat , aku kepingan tulang rusukmu yang hilang . xxbos lara itu , membias tak berbekas","selalu aku perjuangkan sendirian . xxbos aku akan diam , tetapi jemariku akan terus bicara , berteriak tentang keadilan yang kau bungkam . xxbos pada senyum yang sore itu merekah ; sungguh aku tak sengaja jatuh cinta . xxbos kisah lama kembali terkenang lagu lama kembali terngiang nyala lilin mulai surut padam xxbos semesta memberi isyarat , aku kepingan tulang rusukmu yang hilang . xxbos lara itu , membias tak berbekas ,"


Tokenize the text

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df_tokenized = df.apply(lambda row: dls.tokenizer(row['text']), axis=1)

In [ ]:
df_tokenized[0]

(#17) ['xxbos','hanya','karena','sapa','itu','.','kau','tikam','rasamu','.'...]

# N-Gram

In [ ]:
def BiGram(dict, poem):
    for i in range(len(poem)-1):
        word = poem[i]
        next_word  = poem[i+1]
        if(word in dict):
            if(next_word in dict[word]):
                dict[word][next_word] = dict[word][next_word]+1
            else:
                dict[word][next_word] = 1
        else:
            dict[word]={next_word:1}
    return(dict)

In [ ]:
def Next_word(word, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)

In [ ]:
dicBi = {} # The dictionary for the BiGram model
for i in df_tokenized: #Feed it all the poems
    dicBi = BiGram(dicBi, i) #Build the model!

In [ ]:
prev_word = "aku"
generate_str = prev_word
for i in range(40):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str = generate_str +" "+ next_word

In [ ]:
generate_str

'aku tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak'

In [ ]:
prev_word = "senja"
generate_str2 = prev_word
for i in range(40):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str2 = generate_str2 +" "+ next_word

In [ ]:
generate_str2

'senja , aku tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah kau tak pernah'

## MLE

In [ ]:
!pip install -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

In [ ]:
n = 4
train_data, padded_sents = padded_everygram_pipeline(n, df_tokenized)

In [ ]:
model = MLE(n)

In [ ]:
len(model.vocab)

0

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 41715 items>


In [ ]:
len(model.vocab)

41715

In [ ]:
print(model.vocab.lookup(df_tokenized[0]))

('xxbos', 'hanya', 'karena', 'sapa', 'itu', '.', 'kau', 'tikam', 'rasamu', '.', 'sisakan', ',', 'bulir', '-', 'bulir', 'sedu', '.')


In [ ]:
print(model.counts)

<NgramCounter with 4 ngram orders and 8963496 ngrams>


In [ ]:
model.counts['aku']

34977

In [ ]:
model.score('aku')

0.01476093713403093

In [ ]:
model.counts['senja']

5802

In [ ]:
model.score('senja')

0.0024485506833532736

In [ ]:
print(model.generate(100, text_seed=['aku', 'dan'], random_seed=95))

['malam', 'saling', 'berdiam', 'diri', ',', 'enggan', ',', 'membunuh', 'rindu', ',', 'membunuh', 'rindu', 'kita', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, text_seeda, npoem):
    
    
    poems = []
    for i in range(npoem):
      content = []
      content.append(text_seeda)
      for token in model.generate(num_words, text_seed=[text_seeda], random_seed=(i+5)):
          if token == '</s>':
              break
          content.append(token)
      poems.append(detokenize(content))
    return poems

In [ ]:
generate_sent(model,40,'aku', 5)

['aku merindukanmu kala terbangun dari tidurku yang kudapat adalah sapamu ijinkan aku mengenangmu pada dekap hangat tubuh atau rengkuh tanganmu barangkali merelakanmu pergi adalah sebaik - baik air yang membasuh dosa dan khilaf, maka memaafkan adalah sebaik - baiknya usaha',
 'aku tahu kita bukan hikayat rama sinta tapi setidaknya jangan buat aku tambah tidak bergeming . karena aku tahu sejak awal, perihal dirimu beresonansi tanpa tanya, koma, atau pun datang menuju tepi tepat di pesisir kerinduan.',
 'aku ialah daun jatuh . yang tak berdaya membabibuta mencintaimu . apa yang kau suka, duduklah kita akan berbincang lagi tentang riuh dan gemuruh rindu di dadanya yang kering.',
 'aku dapat tertawa bersamamu.',
 'aku masih juga di situ, duduk anggun mendiami sudut hatiku jangan panik apalagi gagu karena rinduku, tidak terburu - buru pergi: mengayuh nyeri sambil sesekali melihat arloji yang berputar ke kiri kudapati senyummu yang kembali aku tak ingin']

In [ ]:
generate_sent(model,40,'senja', 5)

['senja menerpa tubuh kita . begitu syahdu, namun sendu merindukanmu . kasihku . . .',
 'senja tak pernah lupa bahkan luput dari ingatan ini.',
 'senja bersamamu . dalam dekap hangat aku merindumu, aku melamun . rinduku pun lepas, jatuh tergelincir ke dasar ampas . sendiri ia menahan sakit yang membekas.',
 'senja; warnanya seperti pipimu yang merona sehabis ciuman pertama . aku suka.',
 'senja ini aku berpayung cemburu.']